<a href="https://colab.research.google.com/github/Ashujain263648/movie-recommendation-system/blob/main/Movie_Rating_And_Recommendation_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Steps:
1. import the libraries
2. import the dataset
3. Data Analysis
4. Feature Engineering
5. Split the data into two sets using the cv
6. Model selection-KNN
7. training the model
8. Test the model
9. perfornmance- confusion matric


#importing the libraries


In [9]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
import re
import seaborn as sns
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import StratifiedKFold,KFold
from sklearn.preprocessing import StandardScaler

In [10]:
#credits=pd.read_csv('tmdb_5000_credits.csv')
credits = pd.read_csv('tmdb_5000_credits.csv')  # or quoting=3
movies=pd.read_csv('tmdb_5000_movies.csv')


In [11]:
#merge the dataset according to title
movies=movies.merge(credits,on='title')

In [12]:
#remove the unwanted columns
#imp columns(genres,id,keywords,title,overview,cast,crew)
movies = movies[['movie_id','title','overview','genres','keywords','cast','crew']]



In [13]:
# merge the ouverview,genres,keywords,cast,crew in tags
#preprocess the dataset
movies.isnull().sum()# find missing values

,0
movie_id,0
title,0
overview,0
genres,0
keywords,0
cast,0
crew,1


In [14]:
movies.dropna(inplace=True)#remove the misssing values

In [15]:
movies.isnull().sum()

,0
movie_id,0
title,0
overview,0
genres,0
keywords,0
cast,0
crew,0


In [16]:
movies.duplicated().sum()

np.int64(0)

In [17]:
movies.iloc[0].genres

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [18]:
#[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}] conver into list
def convert(obj):
  L=[]
  for i in ast.literal_eval(obj):
    L.append(i['name'])
  return L


In [19]:
import ast
ast.literal_eval('[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]')

[{'id': 28, 'name': 'Action'},
 {'id': 12, 'name': 'Adventure'},
 {'id': 14, 'name': 'Fantasy'},
 {'id': 878, 'name': 'Science Fiction'}]

In [20]:

movies['genres']=movies['genres'].apply(convert)

In [21]:
movies['keywords']=movies['keywords'].apply(convert)

In [22]:
def convert3(obj):
  L=[]
  counter=0
  for i in ast.literal_eval(obj):
    if counter !=3:
       L.append(i['name'])
       counter+=1
    else:
      break
  return L

In [23]:
movies['cast']=movies['cast'].apply(convert3)

In [24]:
def fetch_director(obj):
  L=[]
  for i in ast.literal_eval(obj):
    if i['job']=='Director':
      L.append(i['name'])
      break
  return L


In [25]:
movies['crew']=movies['crew'].apply(fetch_director)

In [26]:
movies['overview'][0] # covert into list

'In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization.'

In [27]:
movies['overview']=movies['overview'].apply(lambda x:x.split())

In [28]:
movies['genres']=movies['genres'].apply(lambda x:[i.replace(" ","") for i in x])
movies['keywords']=movies['keywords'].apply(lambda x:[i.replace(" ","") for i in x])
movies['cast']=movies['cast'].apply(lambda x:[i.replace(" ","") for i in x])
movies['crew']=movies['crew'].apply(lambda x:[i.replace(" ","") for i in x])

In [29]:
movies['tags']=movies['overview']+movies['genres']+movies['keywords']+movies['cast']+movies['crew']

In [ ]:
movies.head(2)

,movie_id,title,overview,genres,keywords,cast,crew,tags
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...","[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron],"[In, the, 22nd, century,, a, paraplegic, Marin..."
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d...","[Adventure, Fantasy, Action]","[ocean, drugabuse, exoticisland, eastindiatrad...","[JohnnyDepp, OrlandoBloom, KeiraKnightley]",[GoreVerbinski],"[Captain, Barbossa,, long, believed, to, be, d..."


In [30]:
new_df=movies[['movie_id','title','tags']]

In [31]:
new_df['tags']=new_df['tags'].apply(lambda x:" ".join(x))

<ipython-input-31-53a061086f2e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags']=new_df['tags'].apply(lambda x:" ".join(x))


In [32]:
new_df['tags']=new_df['tags'].apply(lambda x:x.lower())

<ipython-input-32-242d679aab18>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags']=new_df['tags'].apply(lambda x:x.lower())


In [33]:
#vectorization
from  sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=5000,stop_words='english')

In [34]:
vectors=cv.fit_transform(new_df['tags']).toarray()

In [35]:
cv.get_feature_names_out()

array(['000', '007', '10', ..., 'zoo', 'zooeydeschanel', 'zookeeper'],
      dtype=object)

In [36]:
import nltk

In [37]:
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()

In [38]:
def stem(text):
  y=[]
  for i in text.split():
    y.append(ps.stem(i))
  return " ".join(y)


In [39]:
new_df['tags']=new_df['tags'].apply(stem)

<ipython-input-39-402dd7cb6b10>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags']=new_df['tags'].apply(stem)


In [40]:
from sklearn.metrics.pairwise import cosine_similarity
similarity=cosine_similarity(vectors)

In [42]:
sorted(list(enumerate(similarity[0])),reverse=True,key=lambda x:x[1])[1:6]

[(539, np.float64(0.2347382389307855)),
 (260, np.float64(0.21821789023599236)),
 (582, np.float64(0.2139802462554565)),
 (507, np.float64(0.21052872564651204)),
 (83, np.float64(0.19839002137983247))]

In [52]:
def recommend(movie):
  movie_index=new_df[new_df['title']==movie].index[0]
  distances=similarity[movie_index]
  movies_list= sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[1:6]
  for i in movies_list:
    print(new_df.iloc[i[0]].title)
  return

In [53]:
recommend('Avatar')

Titan A.E.
Ender's Game
Battle: Los Angeles
Independence Day
The Lovers


In [64]:
import pickle

In [65]:
pickle.dump(new_df,open('movies.pkl','wb'))

In [66]:
pickle.dump(new_df.to_dict(),open('movies_dict.pkl','wb'))

In [68]:
pickle.dump(similarity,open('similarity.pkl','wb'))